In [1]:
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
import python_speech_features as mfcc
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import time
import pickle
from sklearn.mixture import GaussianMixture as GMM
import os

In [2]:
source   = "development_set\\"
dest = "speaker_models\\"
train_file = "development_set_enroll_3.txt" # "train.txt" # 
n_files = 3 #training_sample

In [3]:
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc

def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
              first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(audio,rate):
    """on extrait 20 caracteristique de l'audio, et on calcule la variation pour augmenter le nombre de 
    caracteristique a 40"""    
    
    mfcc_feature = mfcc.mfcc(audio,rate, 0.045, 0.01,20,nfft = 2300, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature,delta)) 
    return combined

In [4]:
# Extracting features for each digit data provided by speaker 
file_paths = open(train_file,'r')
count = 1
features = np.asarray(())
for path in file_paths:    
    path = path.strip()   
    print(path)
    
    # read the audio
    sr,audio = read(source + path)
    
    # extract 40 dimensional MFCC & delta MFCC features
    vector   = extract_features(audio,sr)
    
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
    # when features of training files of each digit for a given speaker are concatenated, model training is done
    if count == n_files:    
        gmm = GMM(n_components = 16, max_iter = 200, covariance_type='diag',n_init = 3)
        gmm.fit(features)
        
        # dumping the trained gaussian model
        picklefile = path.split("\\")[0]+".gmm"
        pickle.dump(gmm,open(dest + picklefile,'wb'))
        print('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)    
        features = np.asarray(())
        count = 0
    count = count + 1

sekou_1\audio1.wav
sekou_1\audio2.wav
sekou_1\audio3.wav
+ modeling completed for speaker: sekou_1.gmm  with data point =  (1115, 40)
sekou_2\audio1.wav
sekou_2\audio2.wav
sekou_2\audio3.wav
+ modeling completed for speaker: sekou_2.gmm  with data point =  (1157, 40)
hamadoun_1\audio1.wav
hamadoun_1\audio2.wav
hamadoun_1\audio3.wav
+ modeling completed for speaker: hamadoun_1.gmm  with data point =  (1188, 40)
hamadoun_2\audio1.wav
hamadoun_2\audio2.wav
hamadoun_2\audio3.wav
+ modeling completed for speaker: hamadoun_2.gmm  with data point =  (1188, 40)
djibson_1\audio1.wav
djibson_1\audio2.wav
djibson_1\audio3.wav
+ modeling completed for speaker: djibson_1.gmm  with data point =  (1069, 40)
djibson_2\audio1.wav
djibson_2\audio2.wav
djibson_2\audio3.wav
+ modeling completed for speaker: djibson_2.gmm  with data point =  (1037, 40)


In [5]:
import time, os

test_file = "development_set_test.txt"  # "test.txt" #     

file_paths = open(test_file,'r')

gmm_files = [os.path.join(dest,fname) for fname in os.listdir(dest) if fname.endswith('.gmm')]

gmm_files

['speaker_models\\djibson_1.gmm',
 'speaker_models\\djibson_2.gmm',
 'speaker_models\\hamadoun_1.gmm',
 'speaker_models\\hamadoun_2.gmm',
 'speaker_models\\hassan_1.gmm',
 'speaker_models\\hassan_2.gmm',
 'speaker_models\\ibh_1.gmm',
 'speaker_models\\ibh_2.gmm',
 'speaker_models\\makaveli_1.gmm',
 'speaker_models\\namory_1.gmm',
 'speaker_models\\namory_2.gmm',
 'speaker_models\\sekou_1.gmm',
 'speaker_models\\sekou_2.gmm']

# TEST

In [6]:
#Load the Gaussian Models
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
models

[GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 GaussianMixture(covariance_type='diag', max_iter=200, n_components=16, n_init=3),
 Gau

In [7]:
speakers_digit   = [fname.split("\\")[-1].split(".gmm")[0] for fname in gmm_files]
speakers_digit

['djibson_1',
 'djibson_2',
 'hamadoun_1',
 'hamadoun_2',
 'hassan_1',
 'hassan_2',
 'ibh_1',
 'ibh_2',
 'makaveli_1',
 'namory_1',
 'namory_2',
 'sekou_1',
 'sekou_2']

In [8]:
# Read the test directory and get the list of test audio files 
path_predicted_dict = dict()
for path in file_paths:   
    
    path = path.strip()   
    #print(path)
    sr,audio = read(source + path)
    vector   = extract_features(audio,sr)
    
    log_likelihood = np.zeros(len(models)) 
    
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        cluster=gmm.predict(vector)
        
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()

    
    winner = np.argmax(log_likelihood)
    #print(log_likelihood)
    print(path)
    print("\tSpeaker & Digit detected - ", speakers_digit[winner])
    path_predicted_dict[path] = speakers_digit[winner]

sekou_1\audio4.wav
	Speaker & Digit detected -  sekou_1
sekou_2\audio4.wav
	Speaker & Digit detected -  sekou_2
hamadoun_1\audio4.wav
	Speaker & Digit detected -  hamadoun_1
hamadoun_1\audio5.wav
	Speaker & Digit detected -  hamadoun_1
hamadoun_1\audio6.wav
	Speaker & Digit detected -  hamadoun_1
hamadoun_2\audio4.wav
	Speaker & Digit detected -  hamadoun_2
djibson_1\audio4.wav
	Speaker & Digit detected -  djibson_1
djibson_2\audio4.wav
	Speaker & Digit detected -  ibh_2
test_1\audio4.wav
	Speaker & Digit detected -  ibh_1
test_2\audio4.wav
	Speaker & Digit detected -  ibh_2
ibh_1\audio4.wav
	Speaker & Digit detected -  ibh_1
ibh_2\audio4.wav
	Speaker & Digit detected -  ibh_2


In [9]:
correct_pred = 0
incorrect_pred = 0
for i in path_predicted_dict.keys():
    if i.split("\\")[0] == path_predicted_dict[i]:
        correct_pred += 1
    else:
        incorrect_pred += 1

print("Accuracy on Test Dataset = ", correct_pred*100/(correct_pred + incorrect_pred), "%")

Accuracy on Test Dataset =  75.0 %


In [10]:
import pyaudio
import wave
import os
import pickle
import time
from scipy.io.wavfile import read
from IPython.display import Audio, display, clear_output

In [11]:
 #Voice authentication
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 2

audio = pyaudio.PyAudio()

j=3
while j>=0:
    time.sleep(1.0)
    os.system('cls' if os.name == 'nt' else 'clear')
    print("Dites votre commande dans {} seconds".format(j))
    j-=1

# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
rate=RATE, input=True,
frames_per_buffer=CHUNK)

print("recording...")
frames = []

for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
        
# saving wav file of speaker
waveFile = wave.open('test.wav', 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()
print("Done")
Fs, audio = read("test.wav")
vector =  extract_features(audio,Fs)
log_likelihood = np.zeros(len(models)) 
    
for i in range(len(models)):
    gmm    = models[i]  #checking with each model one by one
    cluster=gmm.predict(vector)
        
    scores = np.array(gmm.score(vector))
    log_likelihood[i] = scores.sum()
    
    
winner = np.argmax(log_likelihood)

result=speakers_digit[winner].split("_")
print("Predicted as:",result[0] )
if(result[1]=="1"):
    print("ouverture de la porte")
else:
    print("validation de la seance")

Dites votre commande dans 3 seconds
Dites votre commande dans 2 seconds
Dites votre commande dans 1 seconds
Dites votre commande dans 0 seconds
recording...
Done
Predicted as: sekou
ouverture de la porte


In [12]:
import math
import contextlib

fname = 'Test-2.wav'
outname = 'filtered.wav'

cutOffFrequency = 400.0

# 
def running_mean(x, windowSize):
  cumsum = np.cumsum(np.insert(x, 0, 0)) 
  return (cumsum[windowSize:] - cumsum[:-windowSize]) / windowSize

# from http://stackov
def interpret_wav(raw_bytes, n_frames, n_channels, sample_width, interleaved = True):

    if sample_width == 1:
        dtype = np.uint8 # unsigned char
    elif sample_width == 2:
        dtype = np.int16 # signed 2-byte short
    else:
        raise ValueError("Only supports 8 and 16 bit audio formats.")

    channels = np.fromstring(raw_bytes, dtype=dtype)

    if interleaved:
        # channels are interleaved, i.e. sample N of channel M follows sample N of channel M-1 in raw data
        channels.shape = (n_frames, n_channels)
        channels = channels.T
    else:
        # channels are not interleaved. All samples from channel M occur before all samples from channel M-1
        channels.shape = (n_channels, n_frames)

    return channels

with contextlib.closing(wave.open(fname,'rb')) as spf:
    sampleRate = spf.getframerate()
    ampWidth = spf.getsampwidth()
    nChannels = spf.getnchannels()
    nFrames = spf.getnframes()

    # Extract Raw Audio from multi-channel Wav File
    signal = spf.readframes(nFrames*nChannels)
    spf.close()
    channels = interpret_wav(signal, nFrames, nChannels, ampWidth, True)

    # get window size
    # from http://dsp.stackexchange.com/questions/9966/what-is-the-cut-off-frequency-of-a-moving-average-filter
    freqRatio = (cutOffFrequency/sampleRate)
    N = int(math.sqrt(0.196196 + freqRatio**2)/freqRatio)

    # Use moviung average (only on first channel)
    filtered = running_mean(channels[0], N).astype(channels.dtype)

    wav_file = wave.open(outname, "w")
    wav_file.setparams((1, ampWidth, sampleRate, nFrames, spf.getcomptype(), spf.getcompname()))
    wav_file.writeframes(filtered.tobytes('C'))
    wav_file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'Test-2.wav'